In [1]:
import pandas as pd

X_full=pd.read_csv('train.csv', index_col='Id')
X_test_full=pd.read_csv('test.csv', index_col='Id')

Again, removing missing values rows in target var and then separate target from predictors

In [2]:
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [3]:
y=X_full.SalePrice

In [5]:
X_full.drop(['SalePrice'], axis=1, inplace=True)

In [6]:
from sklearn.model_selection import train_test_split

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, 
                                                                train_size=0.8, test_size=0.2,random_state=0)

Then, we are cheking cardinality for each column. We are doing next:
1. separate categorical cols with cardinality less then 10
2. separate numerical columns
3. connecting selected data

In [8]:
categorical_col=[cname for cname in X_train_full.columns if
                X_train_full[cname].nunique() < 10 and
                X_train_full[cname].dtype == 'object']

In [9]:
numerical_col=[cname for cname in X_train_full.columns if
              X_train_full[cname].dtype in ['int64','float64']]

In [10]:
my_cols=categorical_col + numerical_col
X_train=X_train_full[my_cols].copy()
X_valid=X_valid_full[my_cols].copy()
X_test=X_test_full[my_cols].copy()

In [12]:
X_train.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,...,774,0,108,0,0,260,0,0,7,2007
871,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,PosN,Norm,...,308,0,0,0,0,0,0,0,8,2009
93,RL,Pave,Grvl,IR1,HLS,AllPub,Inside,Gtl,Norm,Norm,...,432,0,0,44,0,0,0,0,8,2009
818,RL,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Norm,Norm,...,857,150,59,0,0,0,0,0,7,2008
303,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,...,843,468,81,0,0,0,0,0,1,2006


# implementing pipeline

In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [14]:
numerical_transformer=SimpleImputer(strategy='constant')

# for categorical
categorical_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [15]:
# bundle created preprocessing for data

In [17]:
preprocessor=ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_col),
    ('cat', categorical_transformer, categorical_col)
])

Now, we define model.

In [18]:
from sklearn.ensemble import RandomForestRegressor

model=RandomForestRegressor(n_estimators=100, random_state=0)

from sklearn.metrics import mean_absolute_error

my_pipeline=Pipeline(steps=[('preprocessor', preprocessor),
                           ('model',model)])
my_pipeline.fit(X_train,y_train)
preds=my_pipeline.predict(X_valid)
score=mean_absolute_error(y_valid,preds)
print('MAE: ', score)

MAE:  17861.780102739725


# 3. Can we improve performance?

maybe with changing strategies.

In [19]:
numerical_transformer_new=SimpleImputer(strategy='median')

# for categorical
categorical_transformer_new=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [20]:
preprocessor_new=ColumnTransformer(transformers=[
    ('num', numerical_transformer_new, numerical_col),
    ('cat', categorical_transformer_new, categorical_col)
])

model2=RandomForestRegressor(n_estimators=200, random_state=0)

my_pipeline_new=Pipeline(steps=[('preprocessor', preprocessor_new),
                           ('model',model2)])
my_pipeline_new.fit(X_train,y_train)
preds_new=my_pipeline_new.predict(X_valid)
score2=mean_absolute_error(y_valid,preds_new)
print('MAE: ', score2)

MAE:  17385.53157534247


We did little better with placing Simple Imputer based on Median. Can we see by mean?

In [21]:
numerical_transformer_new2=SimpleImputer(strategy='mean')

preprocessor_new2=ColumnTransformer(transformers=[
    ('num', numerical_transformer_new2, numerical_col),
    ('cat', categorical_transformer_new, categorical_col)
])


my_pipeline_new2=Pipeline(steps=[('preprocessor', preprocessor_new2),
                           ('model',model2)])
my_pipeline_new2.fit(X_train,y_train)
preds_new2=my_pipeline_new2.predict(X_valid)
score3=mean_absolute_error(y_valid,preds_new2)
print('MAE: ', score3)


MAE:  17418.883253424658


This is not better!!

So we lower the MAE by changing illing strategy to median and also changing number of estimators to 2000.

Also we can do next:


In [22]:
categorical_transformer_new=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [23]:
preprocessor2=ColumnTransformer(transformers=[
    ('num', numerical_transformer_new, numerical_col),
    ('cat', categorical_transformer_new, categorical_col)
])

In [24]:
my_pipeline2=Pipeline(steps=[('preprocessor', preprocessor2),
                           ('model',model)])
my_pipeline2.fit(X_train,y_train)
preds5=my_pipeline2.predict(X_valid)
score5=mean_absolute_error(y_valid,preds)
print('MAE: ', score5)

MAE:  17861.780102739725


In [25]:
# this is also not good!

Best MAE is given by:

1. strategy - median
2. n_estimators =200
